In [81]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, TimeSeriesSplit

In [56]:
site = pd.read_csv('data/site_info.csv')
energy = pd.read_csv('data/energy.csv')
dangjin_fcst = pd.read_csv('data/add_dangjin_wind_time_insolation.csv')
ulsan_fcst = pd.read_csv('data/add_ulsan_wind_time_insolation.csv')

In [3]:
energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0
...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0,0
25628,2021-01-31 21:00:00,0.0,0.0,0,0
25629,2021-01-31 22:00:00,0.0,0.0,0,0
25630,2021-01-31 23:00:00,0.0,0.0,0,0


In [4]:
dangjin_fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0


In [5]:
site

,Id,Capacity,Address,InstallationAngle,IncidentAngle,Latitude,Longitude
0,당진수상태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
1,당진자재창고태양광,0.7,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
2,당진태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
3,울산태양광,0.5,울산광역시 남구 용잠로 623,20.0,20.0,35.477651,129.380778


In [6]:
c_floating = 1000
c_warehouse = 700
c_dangjin = 1000
c_ulsan = 500

In [7]:
model_path = 'model'
filename = []
filename.append(os.path.join(model_path, 'floating_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'warehouse_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'dangjin_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'ulsan_checkpoint.h5py'))

In [8]:
def make_dataset(data, label, window_size=24):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [9]:
def root_mean_squared_error (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [10]:
df = pd.concat([dangjin_fcst, energy['dangjin_floating']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin_floating
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [11]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin_floating'] / c_floating
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [12]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [13]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16796, 24, 10), (4199, 24, 10), (4541, 24, 10), (672, 24, 10))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
from tensorflow.keras import backend as K

epoch = 50

In [123]:
# model_0 = Sequential()
# model_0.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# model_0.add(Dropout(0.3))
# model_0.add(Dense(1, activation='sigmoid'))
# model_0.compile(loss='mean_squared_error', optimizer='adam')
# model_0.summary()


model_0 = Sequential()
model_0.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_0.add(Dropout(0.3))
model_0.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_0.add(Dense(1, activation='sigmoid'))
model_0.compile(loss='mean_squared_error', optimizer='adam')
model_0.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_5 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 24)                4704      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [124]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_0 = model_0.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
524/526 [============================>.] - ETA: 0s - loss: 0.0392
Epoch 00001: val_loss improved from inf to 0.01657, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 11s 20ms/step - loss: 0.0391 - val_loss: 0.0166
Epoch 2/50
525/526 [============================>.] - ETA: 0s - loss: 0.0139
Epoch 00002: val_loss improved from 0.01657 to 0.01176, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 10s 19ms/step - loss: 0.0139 - val_loss: 0.0118
Epoch 3/50
525/526 [============================>.] - ETA: 0s - loss: 0.0115
Epoch 00003: val_loss improved from 0.01176 to 0.01078, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 10s 18ms/step - loss

Epoch 32/50
525/526 [============================>.] - ETA: 0s - loss: 0.0082
Epoch 00032: val_loss did not improve from 0.00774
526/526 [==============================] - 6s 11ms/step - loss: 0.0082 - val_loss: 0.0080
Epoch 33/50
521/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00033: val_loss did not improve from 0.00774
526/526 [==============================] - 6s 11ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 34/50
525/526 [============================>.] - ETA: 0s - loss: 0.0082
Epoch 00034: val_loss did not improve from 0.00774
526/526 [==============================] - 6s 11ms/step - loss: 0.0082 - val_loss: 0.0084
Epoch 35/50
523/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00035: val_loss did not improve from 0.00774
526/526 [==============================] - 6s 11ms/step - loss: 0.0081 - val_loss: 0.0085
Epoch 36/50
522/526 [============================>.] - ETA: 0s - loss: 0.0082
Epoch 00036: val_loss improved from 0.00774 to

In [125]:
floating_pred = model_0.predict(pred_feature)
floating_pred

array([[5.33032164e-07],
       [3.72745802e-07],
       [3.62714445e-07],
       [6.04707168e-07],
       [1.57530440e-06],
       [8.55383132e-06],
       [1.36852264e-04],
       [5.72466850e-03],
       [9.59913731e-02],
       [2.04554498e-01],
       [3.14379364e-01],
       [3.83238226e-01],
       [4.13631886e-01],
       [3.89954180e-01],
       [3.33095074e-01],
       [2.30356902e-01],
       [8.95924270e-02],
       [7.66181946e-03],
       [1.39036775e-03],
       [2.21043825e-04],
       [3.68139445e-05],
       [7.06923447e-06],
       [1.73223839e-06],
       [6.15353315e-07],
       [3.22643388e-07],
       [2.29453107e-07],
       [2.28391656e-07],
       [3.63840059e-07],
       [1.01062540e-06],
       [8.91053878e-06],
       [1.56342983e-04],
       [9.28398967e-03],
       [2.16572165e-01],
       [4.10494357e-01],
       [5.52178860e-01],
       [6.15268707e-01],
       [6.34682715e-01],
       [6.01525664e-01],
       [4.94549721e-01],
       [3.17058921e-01],


In [18]:
df = pd.concat([dangjin_fcst, energy['dangjin_warehouse']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin_warehouse
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [19]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin_warehouse'] / c_warehouse
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [20]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [21]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16776, 24, 10), (4195, 24, 10), (4541, 24, 10), (672, 24, 10))

In [119]:
# model_1 = Sequential()
# model_1.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# model_1.add(Dropout(0.1))
# model_1.add(Dense(1, activation='sigmoid'))
# model_1.compile(loss='mean_squared_error', optimizer='adam')
# model_1.summary()

model_1 = Sequential()
model_1.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_1.add(Dropout(0.3))
model_1.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss='mean_squared_error', optimizer='adam')
model_1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 24)                4704      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [120]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[1], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_1 = model_1.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50


Exception ignored in: <function Booster.__del__ at 0x0000029ED67B9AF8>
Traceback (most recent call last):
  File "D:\Anaconda\envs\data_engeneering\lib\site-packages\lightgbm\basic.py", line 2288, in __del__
    _safe_call(_LIB.LGBM_BoosterFree(self.handle))
OSError: exception: access violation reading 0xFFFFFFFFFFFFFFFF


524/526 [============================>.] - ETA: 0s - loss: 0.0353
Epoch 00001: val_loss improved from inf to 0.01581, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
526/526 [==============================] - 10s 19ms/step - loss: 0.0353 - val_loss: 0.0158
Epoch 2/50
523/526 [============================>.] - ETA: 0s - loss: 0.0132
Epoch 00002: val_loss improved from 0.01581 to 0.01148, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
526/526 [==============================] - 10s 19ms/step - loss: 0.0132 - val_loss: 0.0115
Epoch 3/50
523/526 [============================>.] - ETA: 0s - loss: 0.0109
Epoch 00003: val_loss improved from 0.01148 to 0.00977, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
526/526 [==============================] - 9s 17ms/step - loss: 0.01

Epoch 32/50
526/526 [==============================] - ETA: 0s - loss: 0.0081
Epoch 00032: val_loss did not improve from 0.00772
526/526 [==============================] - 5s 10ms/step - loss: 0.0081 - val_loss: 0.0078
Epoch 33/50
522/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00033: val_loss did not improve from 0.00772
526/526 [==============================] - 5s 10ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 34/50
526/526 [==============================] - ETA: 0s - loss: 0.0082
Epoch 00034: val_loss did not improve from 0.00772
526/526 [==============================] - 6s 10ms/step - loss: 0.0082 - val_loss: 0.0078
Epoch 35/50
526/526 [==============================] - ETA: 0s - loss: 0.0081
Epoch 00035: val_loss did not improve from 0.00772
526/526 [==============================] - 5s 10ms/step - loss: 0.0081 - val_loss: 0.0077
Epoch 36/50
524/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00036: val_loss did not improve from 0.0

In [121]:
warehouse_pred = model_1.predict(pred_feature)

In [122]:
warehouse_pred

array([[2.25782394e-04],
       [2.70098448e-04],
       [2.84165144e-04],
       [2.83449888e-04],
       [3.50594521e-04],
       [5.11527061e-04],
       [1.01897120e-03],
       [3.53920460e-03],
       [2.44227052e-02],
       [9.56624448e-02],
       [1.93184227e-01],
       [2.55661309e-01],
       [2.84423947e-01],
       [2.84329116e-01],
       [2.37586707e-01],
       [1.55447304e-01],
       [4.61247861e-02],
       [6.35817647e-03],
       [2.34067440e-03],
       [6.47902489e-04],
       [2.58982182e-04],
       [1.31785870e-04],
       [1.51187181e-04],
       [1.95384026e-04],
       [2.61187553e-04],
       [2.51621008e-04],
       [2.11089849e-04],
       [2.41339207e-04],
       [4.03523445e-04],
       [8.06659460e-04],
       [2.97683477e-03],
       [3.19654942e-02],
       [1.92036062e-01],
       [3.38903010e-01],
       [4.66577530e-01],
       [5.33016503e-01],
       [5.43633759e-01],
       [4.94075745e-01],
       [4.05828983e-01],
       [2.74150014e-01],


In [26]:
df = pd.concat([dangjin_fcst, energy['dangjin']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [27]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin'] / c_dangjin
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [28]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [29]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16815, 24, 10), (4204, 24, 10), (4541, 24, 10), (672, 24, 10))

In [30]:
model_2 = Sequential()
model_2.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_2.add(Dropout(0.3))
model_2.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(loss='mean_squared_error', optimizer='adam')
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 24)                4704      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [31]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[2], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_2 = model_2.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
524/526 [============================>.] - ETA: 0s - loss: 0.0377
Epoch 00001: val_loss improved from inf to 0.01441, saving model to model\dangjin_checkpoint.h5py
INFO:tensorflow:Assets written to: model\dangjin_checkpoint.h5py\assets
526/526 [==============================] - 10s 19ms/step - loss: 0.0376 - val_loss: 0.0144
Epoch 2/50
524/526 [============================>.] - ETA: 0s - loss: 0.0116
Epoch 00002: val_loss improved from 0.01441 to 0.01007, saving model to model\dangjin_checkpoint.h5py
INFO:tensorflow:Assets written to: model\dangjin_checkpoint.h5py\assets
526/526 [==============================] - 9s 17ms/step - loss: 0.0116 - val_loss: 0.0101
Epoch 3/50
525/526 [============================>.] - ETA: 0s - loss: 0.0097
Epoch 00003: val_loss did not improve from 0.01007
526/526 [==============================] - 7s 13ms/step - loss: 0.0097 - val_loss: 0.0110
Epoch 4/50
525/526 [============================>.] - ETA: 0s - loss: 0.0088
Epoch 00004: val_loss impr

In [32]:
dangjin_pred = model_2.predict(pred_feature)

In [33]:
dangjin_pred

array([[5.65842100e-26],
       [4.92706180e-28],
       [9.52506113e-28],
       [4.33447513e-26],
       [3.75665156e-23],
       [6.71972942e-16],
       [1.23902899e-09],
       [2.91869273e-05],
       [3.82897258e-03],
       [3.05702388e-02],
       [9.87871289e-02],
       [1.87104762e-01],
       [2.70812809e-01],
       [3.10654879e-01],
       [2.78197825e-01],
       [1.66502297e-01],
       [2.50906348e-02],
       [1.19360826e-04],
       [5.31847309e-06],
       [4.61297596e-08],
       [3.59941521e-11],
       [2.65756248e-15],
       [3.58806134e-20],
       [1.22017800e-24],
       [1.99804793e-27],
       [1.80545003e-28],
       [1.02993271e-26],
       [1.26914806e-23],
       [6.38015352e-21],
       [2.69343440e-13],
       [5.66222752e-07],
       [1.46022439e-03],
       [9.99324918e-02],
       [2.71261990e-01],
       [4.50363517e-01],
       [5.69824636e-01],
       [6.24525845e-01],
       [6.18950486e-01],
       [5.11442125e-01],
       [3.00131738e-01],


In [34]:
df = pd.concat([dangjin_fcst, energy['ulsan']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,ulsan
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [35]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['ulsan'] / c_ulsan
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [36]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16815, 24, 10), (4204, 24, 10), (4541, 24, 10), (672, 24, 10))

In [38]:
model_3 = Sequential()
model_3.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_3.add(Dropout(0.3))
model_3.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(loss='mean_squared_error', optimizer='adam')
model_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 24)                4704      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [39]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[3], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_3 = model_3.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
523/526 [============================>.] - ETA: 0s - loss: 0.0367
Epoch 00001: val_loss improved from inf to 0.01543, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 9s 18ms/step - loss: 0.0365 - val_loss: 0.0154
Epoch 2/50
525/526 [============================>.] - ETA: 0s - loss: 0.0138
Epoch 00002: val_loss improved from 0.01543 to 0.01171, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 9s 17ms/step - loss: 0.0138 - val_loss: 0.0117
Epoch 3/50
523/526 [============================>.] - ETA: 0s - loss: 0.0117
Epoch 00003: val_loss improved from 0.01171 to 0.01124, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 9s 17ms/step - loss: 0.0117 - val_loss: 

In [40]:
ulsan_pred = model_3.predict(pred_feature)

In [41]:
ulsan_pred

array([[5.3951144e-04],
       [6.6477060e-04],
       [1.0405481e-03],
       [1.9873977e-03],
       [4.2705536e-03],
       [9.3585551e-03],
       [1.9685447e-02],
       [4.0504217e-02],
       [1.1434302e-01],
       [2.1249732e-01],
       [2.9405293e-01],
       [3.7360251e-01],
       [4.2595834e-01],
       [4.1856194e-01],
       [3.6597294e-01],
       [2.8460032e-01],
       [1.6140354e-01],
       [4.9769193e-02],
       [2.2674829e-02],
       [8.9971423e-03],
       [3.4216344e-03],
       [1.2761950e-03],
       [6.5413117e-04],
       [3.9273500e-04],
       [3.7491322e-04],
       [4.6670437e-04],
       [7.8079104e-04],
       [1.6247928e-03],
       [3.8490891e-03],
       [9.2055798e-03],
       [2.0782381e-02],
       [5.2879721e-02],
       [2.0804134e-01],
       [3.5421196e-01],
       [5.0075662e-01],
       [5.5518806e-01],
       [5.5367601e-01],
       [5.2226543e-01],
       [4.6098691e-01],
       [3.5996646e-01],
       [2.2994557e-01],
       [8.716717

## LGBM

In [42]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [61]:
def make_train_data(energy_df, fcst_df, target):
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df.loc[:25608-1]
    fcst.index = range(fcst.shape[0])
    
    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[[#'year', 'month', 'day', 'hour', 
                            'Year sin', 'Year cos', 'Day sin', 'Day cos',
                            'Temperature', 'Humidity', 
                            #'WindSpeed', 'WindDirection',
                            'Wind_X', 'Wind_Y',
                            'azimuth', 'elevation', 'insolation',
                            'Cloud', target]]


    x_df = feature_df.loc[:, :'Cloud']
    y_df = feature_df[target]

    return x_df, y_df

In [62]:
def train_dataset(energy_df, fcst_df, target):
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df.loc[:25608-1]
    fcst.index = range(fcst.shape[0])
    
    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[[#'year', 'month', 'day', 'hour', 
                            'Year sin', 'Year cos', 'Day sin', 'Day cos',
                            'Temperature', 'Humidity', 
                            #'WindSpeed', 'WindDirection',
                            'Wind_X', 'Wind_Y',
                            'azimuth', 'elevation', 'insolation',
                            'Cloud', target]]

    # 마지막 30일을 검증데이터셋으로 나머지를 학습 데이터셋으로 선택
    train_df = feature_df.iloc[:-24*30]
    val_df = feature_df.iloc[-24*30:]
    
    # 발전량이 0인 데이터를 제외
    train_df = train_df[train_df[target]!=0]
    
    train_x = train_df.loc[:, :'Cloud'].to_numpy()
    train_y = train_df[target].to_numpy()
    
    val_x = val_df.loc[:, :'Cloud'].to_numpy()
    val_y = val_df[target].to_numpy()
    
    return train_x, train_y, val_x, val_y

In [63]:
def nmae_10(y_pred, dataset):
    y_true = dataset.get_label()
    
    absolute_error = abs(y_true - y_pred)
    absolute_error /= capacity
    
    target_idx = np.where(y_true>=capacity*0.1)
    
    nmae = 100 * absolute_error[target_idx].mean()
    
    return 'score', nmae, False

In [64]:
def sola_nmae(answer, pred):
    absolute_error = np.abs(answer - pred)
    
    absolute_error /= capacity
    
    target_idx = np.where(answer>=capacity*0.1)
    
    nmae = 100 * absolute_error[target_idx].mean()
    
    return nmae

In [65]:
# 기본 패러미터
params_default = {
    'seed':42,
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric':'mae',
    'num_iterations': 10000,
    'learning_rate': 0.09,
    # 'max_depth': 16,
    # 'num_leaves': 39000,
    # 'bagging_freq': 2,
    # 'bagging_fraction': 0.7,
    # 'feature_fraction': 0.8,
}

In [66]:
def objectiveLGBM(trial: Trial, energy, fcst, target):
    params = {
        'verbose': -1,
        'seed': 2021,
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric':'mse',
        'learning_rate': 0.09,
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'num_leaves': trial.suggest_int('num_leaves', 2, 65535),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 5),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 1.5),
        'max_bin': trial.suggest_int('max_bin', 1, 1000)
    }

    train_x, train_y, val_x, val_y = train_dataset(energy, fcst, target)
    train_ds = lgb.Dataset(train_x, train_y)
    val_ds = lgb.Dataset(val_x, val_y)

    model = lgb.train(params=params, train_set=train_ds, num_boost_round=1000, valid_sets=val_ds, verbose_eval=50, early_stopping_rounds=30, feval=nmae_10)

    pred = model.predict(val_x)
    score = sola_nmae(val_y, pred)

    return score

In [67]:
def paramsTuning(energy, fcst, target):
    sampler = TPESampler()
    study = optuna.create_study(
        study_name="lgbm_parameter_opt",
        direction="minimize",
        sampler=sampler,
    )
    study.optimize(lambda trial: objectiveLGBM(trial, energy, fcst, target), n_trials=10)
    print()
    print("Best Score:", study.best_value)
    print("Best trial:", study.best_trial.params)
    return study

In [79]:
def kfold_lgbm(params, n_split, energy, fcst, test_set, target):
    cv_score = 0
    cv_predict = np.zeros((test_set.shape[0]), dtype=float)
    x_df, y_df = make_train_data(energy, fcst, target)

    folds = KFold(n_splits=n_split, shuffle=True, random_state=42)
    splits = folds.split(x_df, y_df)

    for index, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {index} ============\n")
        train_x, val_x = x_df.iloc[train_idx].to_numpy(), x_df.iloc[valid_idx].to_numpy()
        train_y, val_y = y_df.iloc[train_idx].to_numpy(), y_df.iloc[valid_idx].to_numpy()

        # # 발전량이 0인 데이터를 제외
        # train_x = train_x[train_y[target]!=0]
        # train_y = train_y[train_y[target]!=0]

        train_ds = lgb.Dataset(train_x, train_y)
        val_ds = lgb.Dataset(val_x, val_y)

        model = lgb.train(params=params, train_set=train_ds, num_boost_round=1000, valid_sets=val_ds, verbose_eval=500, early_stopping_rounds=30, feval=nmae_10)

        cv_score += sola_nmae(val_y, model.predict(val_x))
        cv_predict += model.predict(test_set)

    print('CV Mean Score : ', cv_score/n_split)
    predict = cv_predict/n_split

    return predict
    

In [71]:
# 당진수상태양광
capacity = 1000

study = paramsTuning(energy, dangjin_fcst, target='dangjin_floating')

params_floating = params_default.copy()
params_floating.update(study.best_trial.params)
print(params_floating)

[I 2021-06-25 18:20:13,959] A new study created in memory with name: lgbm_parameter_opt


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 4273.73	valid_0's score: 8.14209
Early stopping, best iteration is:
[51]	valid_0's l2: 4264.76	valid_0's score: 8.11577


[I 2021-06-25 18:20:23,185] Trial 0 finished with value: 8.115767911329602 and parameters: {'max_depth': 37, 'num_leaves': 42280, 'min_data_in_leaf': 125, 'lambda_l1': 7.751274561481715, 'lambda_l2': 9.695103605634351, 'bagging_freq': 0, 'bagging_fraction': 0.6882151994364001, 'feature_fraction': 0.46300841062783304, 'scale_pos_weight': 1.093830701218132, 'max_bin': 137}. Best is trial 0 with value: 8.115767911329602.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 4602.03	valid_0's score: 8.58582
[100]	valid_0's l2: 4600.07	valid_0's score: 8.37137
Early stopping, best iteration is:
[73]	valid_0's l2: 4506.51	valid_0's score: 8.42449


[I 2021-06-25 18:20:45,919] Trial 1 finished with value: 8.424488767331951 and parameters: {'max_depth': 33, 'num_leaves': 47937, 'min_data_in_leaf': 131, 'lambda_l1': 7.675819622070278, 'lambda_l2': 6.277079231939956, 'bagging_freq': 3, 'bagging_fraction': 0.1893281165574212, 'feature_fraction': 0.3791874297105209, 'scale_pos_weight': 1.2683936350831821, 'max_bin': 878}. Best is trial 0 with value: 8.115767911329602.


...

Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 5273.95	valid_0's score: 8.8295
Early stopping, best iteration is:
[68]	valid_0's l2: 5036.74	valid_0's score: 8.4932


[I 2021-06-25 18:23:03,623] Trial 8 finished with value: 8.493198183494924 and parameters: {'max_depth': 46, 'num_leaves': 60014, 'min_data_in_leaf': 157, 'lambda_l1': 5.623916308164326, 'lambda_l2': 7.597334561768402, 'bagging_freq': 5, 'bagging_fraction': 0.22296420863019795, 'feature_fraction': 0.2516754991876165, 'scale_pos_weight': 1.140287509240676, 'max_bin': 741}. Best is trial 4 with value: 7.883837192169824.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 4191.89	valid_0's score: 8.14572
Early stopping, best iteration is:
[59]	valid_0's l2: 4154.72	valid_0's score: 8.06514


[I 2021-06-25 18:23:44,373] Trial 9 finished with value: 8.065137745285494 and parameters: {'max_depth': 28, 'num_leaves': 60103, 'min_data_in_leaf': 132, 'lambda_l1': 5.186016779989053, 'lambda_l2': 2.539486993634117, 'bagging_freq': 2, 'bagging_fraction': 0.7015616725321889, 'feature_fraction': 0.963814197025767, 'scale_pos_weight': 1.2277040459098254, 'max_bin': 869}. Best is trial 4 with value: 7.883837192169824.



Best Score: 7.883837192169824
Best trial: {'max_depth': 44, 'num_leaves': 22541, 'min_data_in_leaf': 156, 'lambda_l1': 4.730737437205439, 'lambda_l2': 6.159114807290155, 'bagging_freq': 4, 'bagging_fraction': 0.631364526462678, 'feature_fraction': 0.32871324631817334, 'scale_pos_weight': 1.2963445042796184, 'max_bin': 405}
{'seed': 42, 'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'num_iterations': 10000, 'learning_rate': 0.09, 'max_depth': 44, 'num_leaves': 22541, 'min_data_in_leaf': 156, 'lambda_l1': 4.730737437205439, 'lambda_l2': 6.159114807290155, 'bagging_freq': 4, 'bagging_fraction': 0.631364526462678, 'feature_fraction': 0.32871324631817334, 'scale_pos_weight': 1.2963445042796184, 'max_bin': 405}


In [72]:
# 당진자재창고태양광
capacity = 700

study = paramsTuning(energy, dangjin_fcst, target='dangjin_warehouse')

params_warehouse = params_default.copy()
params_warehouse.update(study.best_trial.params)
print(params_warehouse)

[I 2021-06-25 18:23:44,404] A new study created in memory with name: lgbm_parameter_opt


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 2910.13	valid_0's score: 10.0634
Early stopping, best iteration is:
[54]	valid_0's l2: 2898.99	valid_0's score: 10.0687


[I 2021-06-25 18:24:30,152] Trial 0 finished with value: 10.068695802053568 and parameters: {'max_depth': 31, 'num_leaves': 65468, 'min_data_in_leaf': 114, 'lambda_l1': 7.294911395984758, 'lambda_l2': 2.757276140653194, 'bagging_freq': 3, 'bagging_fraction': 0.8125333994138862, 'feature_fraction': 0.39540638384089477, 'scale_pos_weight': 1.2720175736022057, 'max_bin': 584}. Best is trial 0 with value: 10.068695802053568.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 2792.17	valid_0's score: 10.3739
Early stopping, best iteration is:
[67]	valid_0's l2: 2758.78	valid_0's score: 10.2197

[I 2021-06-25 18:24:30,446] Trial 1 finished with value: 10.219707544011795 and parameters: {'max_depth': 9, 'num_leaves': 8175, 'min_data_in_leaf': 51, 'lambda_l1': 6.113945333696298, 'lambda_l2': 8.818079212848568, 'bagging_freq': 1, 'bagging_fraction': 0.9677763855246322, 'feature_fraction': 0.8980534585157102, 'scale_pos_weight': 1.1118098018891518, 'max_bin': 285}. Best is trial 0 with value: 10.068695802053568.


...

Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 2873.92	valid_0's score: 9.92022
Early stopping, best iteration is:
[69]	valid_0's l2: 2800.79	valid_0's score: 9.7992


[I 2021-06-25 18:25:03,837] Trial 7 finished with value: 9.799203883784116 and parameters: {'max_depth': 19, 'num_leaves': 11078, 'min_data_in_leaf': 85, 'lambda_l1': 3.609634882450389, 'lambda_l2': 8.76124188309798, 'bagging_freq': 0, 'bagging_fraction': 0.406463963049823, 'feature_fraction': 0.38117606660608727, 'scale_pos_weight': 1.4867351880354245, 'max_bin': 651}. Best is trial 2 with value: 9.507144986975016.
[I 2021-06-25 18:25:04,031] Trial 8 finished with value: 10.153951941998875 and parameters: {'max_depth': 26, 'num_leaves': 1530, 'min_data_in_leaf': 118, 'lambda_l1': 4.458791593500169, 'lambda_l2': 3.318642813362006, 'bagging_freq': 1, 'bagging_fraction': 0.7492656099120859, 'feature_fraction': 0.4112988171686429, 'scale_pos_weight': 1.3488474248848377, 'max_bin': 128}. Best is trial 2 with value: 9.507144986975016.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 2929.9	valid_0's score: 10.1842
Early stopping, best iteration is:
[41]	valid_0's l2: 2992.34	valid_0's score: 10.154
Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 2956.85	valid_0's score: 10.2681
Early stopping, best iteration is:
[60]	valid_0's l2: 2930.32	valid_0's score: 10.1628


[I 2021-06-25 18:25:24,734] Trial 9 finished with value: 10.162763826202077 and parameters: {'max_depth': 17, 'num_leaves': 37338, 'min_data_in_leaf': 46, 'lambda_l1': 7.967932259786963, 'lambda_l2': 2.5861800846565397, 'bagging_freq': 3, 'bagging_fraction': 0.6522863130927123, 'feature_fraction': 0.4628124579755556, 'scale_pos_weight': 1.3664146283280678, 'max_bin': 814}. Best is trial 2 with value: 9.507144986975016.



Best Score: 9.507144986975016
Best trial: {'max_depth': 35, 'num_leaves': 17327, 'min_data_in_leaf': 55, 'lambda_l1': 3.0298103359316575, 'lambda_l2': 0.21324763811625402, 'bagging_freq': 2, 'bagging_fraction': 0.477713708753434, 'feature_fraction': 0.30038019356889367, 'scale_pos_weight': 1.046253622487332, 'max_bin': 259}
{'seed': 42, 'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'num_iterations': 10000, 'learning_rate': 0.09, 'max_depth': 35, 'num_leaves': 17327, 'min_data_in_leaf': 55, 'lambda_l1': 3.0298103359316575, 'lambda_l2': 0.21324763811625402, 'bagging_freq': 2, 'bagging_fraction': 0.477713708753434, 'feature_fraction': 0.30038019356889367, 'scale_pos_weight': 1.046253622487332, 'max_bin': 259}


In [73]:
# 당진태양광
capacity = 1000

study = paramsTuning(energy, dangjin_fcst, target='dangjin')

params_dangjin = params_default.copy()
params_dangjin.update(study.best_trial.params)
print(params_dangjin)

[I 2021-06-25 18:25:24,755] A new study created in memory with name: lgbm_parameter_opt


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 6726.63	valid_0's score: 10.0763
Early stopping, best iteration is:
[35]	valid_0's l2: 7063.88	valid_0's score: 9.93544


[I 2021-06-25 18:25:25,034] Trial 0 finished with value: 9.935439081248049 and parameters: {'max_depth': 8, 'num_leaves': 23389, 'min_data_in_leaf': 23, 'lambda_l1': 1.1476478646442145, 'lambda_l2': 5.525583149714635, 'bagging_freq': 4, 'bagging_fraction': 0.757872465003774, 'feature_fraction': 0.6724180670743577, 'scale_pos_weight': 1.471194469535765, 'max_bin': 416}. Best is trial 0 with value: 9.935439081248049.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 6876.89	valid_0's score: 10.257
Early stopping, best iteration is:
[52]	valid_0's l2: 6857	valid_0's score: 10.2799


[I 2021-06-25 18:25:26,472] Trial 1 finished with value: 10.279930644883805 and parameters: {'max_depth': 42, 'num_leaves': 3781, 'min_data_in_leaf': 70, 'lambda_l1': 3.9327608952195825, 'lambda_l2': 7.647095441152811, 'bagging_freq': 5, 'bagging_fraction': 0.8098095154979915, 'feature_fraction': 0.6822339509699329, 'scale_pos_weight': 1.1154503508202234, 'max_bin': 553}. Best is trial 0 with value: 9.935439081248049.


...

Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 6563.8	valid_0's score: 10.0894
Early stopping, best iteration is:
[57]	valid_0's l2: 6480.37	valid_0's score: 9.88187


[I 2021-06-25 18:27:22,579] Trial 7 finished with value: 9.881870155682684 and parameters: {'max_depth': 36, 'num_leaves': 3312, 'min_data_in_leaf': 36, 'lambda_l1': 0.07972861482175242, 'lambda_l2': 7.181434370404548, 'bagging_freq': 5, 'bagging_fraction': 0.6768568153777087, 'feature_fraction': 0.7929980060910237, 'scale_pos_weight': 1.0241245605620435, 'max_bin': 474}. Best is trial 7 with value: 9.881870155682684.
[I 2021-06-25 18:27:22,711] Trial 8 finished with value: 10.292529213526933 and parameters: {'max_depth': 26, 'num_leaves': 4089, 'min_data_in_leaf': 138, 'lambda_l1': 6.473748323867521, 'lambda_l2': 3.7347669319797863, 'bagging_freq': 4, 'bagging_fraction': 0.2818353460141151, 'feature_fraction': 0.5996001031903712, 'scale_pos_weight': 1.3358215277783148, 'max_bin': 29}. Best is trial 7 with value: 9.881870155682684.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 6895.78	valid_0's score: 10.3517
Early stopping, best iteration is:
[52]	valid_0's l2: 6821.68	valid_0's score: 10.2925
Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 7403.52	valid_0's score: 10.4662
[100]	valid_0's l2: 7030.36	valid_0's score: 10.4206
Early stopping, best iteration is:
[82]	valid_0's l2: 7082.57	valid_0's score: 10.172


[I 2021-06-25 18:27:51,453] Trial 9 finished with value: 10.172031204893523 and parameters: {'max_depth': 35, 'num_leaves': 43985, 'min_data_in_leaf': 132, 'lambda_l1': 4.5037376776376465, 'lambda_l2': 4.765603319392345, 'bagging_freq': 2, 'bagging_fraction': 0.22434161641571781, 'feature_fraction': 0.8769970632668871, 'scale_pos_weight': 1.1049429776057051, 'max_bin': 635}. Best is trial 7 with value: 9.881870155682684.



Best Score: 9.881870155682684
Best trial: {'max_depth': 36, 'num_leaves': 3312, 'min_data_in_leaf': 36, 'lambda_l1': 0.07972861482175242, 'lambda_l2': 7.181434370404548, 'bagging_freq': 5, 'bagging_fraction': 0.6768568153777087, 'feature_fraction': 0.7929980060910237, 'scale_pos_weight': 1.0241245605620435, 'max_bin': 474}
{'seed': 42, 'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'num_iterations': 10000, 'learning_rate': 0.09, 'max_depth': 36, 'num_leaves': 3312, 'min_data_in_leaf': 36, 'lambda_l1': 0.07972861482175242, 'lambda_l2': 7.181434370404548, 'bagging_freq': 5, 'bagging_fraction': 0.6768568153777087, 'feature_fraction': 0.7929980060910237, 'scale_pos_weight': 1.0241245605620435, 'max_bin': 474}


In [74]:
# 울산태양광
capacity = 500

study = paramsTuning(energy, ulsan_fcst, target='ulsan')

params_ulsan = params_default.copy()
params_ulsan.update(study.best_trial.params)
print(params_ulsan)

[I 2021-06-25 18:27:51,464] A new study created in memory with name: lgbm_parameter_opt


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 962.55	valid_0's score: 6.70775
[100]	valid_0's l2: 899.425	valid_0's score: 6.44906
Early stopping, best iteration is:
[115]	valid_0's l2: 887.83	valid_0's score: 6.38196


[I 2021-06-25 18:28:04,694] Trial 0 finished with value: 6.3819566834501895 and parameters: {'max_depth': 15, 'num_leaves': 33356, 'min_data_in_leaf': 28, 'lambda_l1': 1.367480142725338, 'lambda_l2': 1.6946707548589202, 'bagging_freq': 1, 'bagging_fraction': 0.6883789620698828, 'feature_fraction': 0.2909784284348658, 'scale_pos_weight': 1.3721249414208094, 'max_bin': 181}. Best is trial 0 with value: 6.3819566834501895.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 699.149	valid_0's score: 5.98477
Early stopping, best iteration is:
[56]	valid_0's l2: 681.225	valid_0's score: 5.84766


[I 2021-06-25 18:28:07,277] Trial 1 finished with value: 5.847660942154398 and parameters: {'max_depth': 39, 'num_leaves': 35007, 'min_data_in_leaf': 24, 'lambda_l1': 7.259917067347333, 'lambda_l2': 1.2432596802435392, 'bagging_freq': 4, 'bagging_fraction': 0.792951924961241, 'feature_fraction': 0.96388185161154, 'scale_pos_weight': 1.2003962884131039, 'max_bin': 85}. Best is trial 1 with value: 5.847660942154398.


...

Early stopping, best iteration is:
[45]	valid_0's l2: 772.297	valid_0's score: 5.80913
Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 703.587	valid_0's score: 5.77621
Early stopping, best iteration is:
[46]	valid_0's l2: 700.091	valid_0's score: 5.73814


[I 2021-06-25 18:29:20,462] Trial 7 finished with value: 5.738137296497059 and parameters: {'max_depth': 18, 'num_leaves': 19634, 'min_data_in_leaf': 22, 'lambda_l1': 1.8026678174907829, 'lambda_l2': 8.005961934328953, 'bagging_freq': 1, 'bagging_fraction': 0.8412825217309254, 'feature_fraction': 0.8686931524781989, 'scale_pos_weight': 1.2792203247015936, 'max_bin': 105}. Best is trial 4 with value: 5.669116933397492.
[I 2021-06-25 18:29:20,608] Trial 8 finished with value: 6.09870417639937 and parameters: {'max_depth': 5, 'num_leaves': 18542, 'min_data_in_leaf': 155, 'lambda_l1': 6.368173928853674, 'lambda_l2': 6.5991504212668275, 'bagging_freq': 3, 'bagging_fraction': 0.4229605579433634, 'feature_fraction': 0.38962153952457246, 'scale_pos_weight': 1.0987630655458762, 'max_bin': 173}. Best is trial 4 with value: 5.669116933397492.


Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 842.934	valid_0's score: 6.40137
[100]	valid_0's l2: 831.297	valid_0's score: 6.10306
Early stopping, best iteration is:
[79]	valid_0's l2: 816.76	valid_0's score: 6.0987
Training until validation scores don't improve for 30 rounds
[50]	valid_0's l2: 755.381	valid_0's score: 5.95116
Early stopping, best iteration is:
[59]	valid_0's l2: 741.411	valid_0's score: 5.92561


[I 2021-06-25 18:29:27,960] Trial 9 finished with value: 5.92560759311106 and parameters: {'max_depth': 27, 'num_leaves': 15186, 'min_data_in_leaf': 172, 'lambda_l1': 8.464881015794175, 'lambda_l2': 6.923638122837476, 'bagging_freq': 2, 'bagging_fraction': 0.7243046353314126, 'feature_fraction': 0.821598360864456, 'scale_pos_weight': 1.2417493420413312, 'max_bin': 441}. Best is trial 4 with value: 5.669116933397492.



Best Score: 5.669116933397492
Best trial: {'max_depth': 12, 'num_leaves': 36512, 'min_data_in_leaf': 49, 'lambda_l1': 2.0280770580408145, 'lambda_l2': 1.797849646450339, 'bagging_freq': 4, 'bagging_fraction': 0.9960181980973725, 'feature_fraction': 0.7282043631673161, 'scale_pos_weight': 1.3382632586705563, 'max_bin': 155}
{'seed': 42, 'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'num_iterations': 10000, 'learning_rate': 0.09, 'max_depth': 12, 'num_leaves': 36512, 'min_data_in_leaf': 49, 'lambda_l1': 2.0280770580408145, 'lambda_l2': 1.797849646450339, 'bagging_freq': 4, 'bagging_fraction': 0.9960181980973725, 'feature_fraction': 0.7282043631673161, 'scale_pos_weight': 1.3382632586705563, 'max_bin': 155}


In [76]:
def test_datast(fcst_df):
    start = '2021-02-01 00:00:00'
    end = '2021-02-28 23:00:00'
    
    start_idx = fcst_df[fcst_df['Forecast_time']==start].index[0]
    end_idx = fcst_df[fcst_df['Forecast_time']==end].index[0]
    
    test_df = fcst_df.loc[start_idx:end_idx, :].copy()
    
    test_df = test_df[[ #'year', 'month', 'day', 'hour', 
                        'Year sin', 'Year cos', 'Day sin', 'Day cos',
                        'Temperature', 'Humidity', 
                        #'WindSpeed', 'WindDirection',
                        'Wind_X', 'Wind_Y',
                        'azimuth', 'elevation', 'insolation',
                        'Cloud']]
    
    test_x = test_df.to_numpy()
    
    return test_x

In [77]:
dangjin_test = test_datast(dangjin_fcst)
ulsan_test = test_datast(ulsan_fcst)

In [82]:
# 당진수상태양광 예측 모델 학습
capacity = 1000
LGBM_dangjin_floating_pred = kfold_lgbm(params_floating, 5, energy, dangjin_fcst, dangjin_test, target='dangjin_floating')

============ Fold 0 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3989


D:\Anaconda\envs\data_engeneering\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 122.021625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iter

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [83]:
# 당진자재창고태양광 예측 모델 학습
capacity = 700
LGBM_dangjin_warehouse_pred = kfold_lgbm(params_warehouse, 5, energy, dangjin_fcst, dangjin_test, target='dangjin_warehouse')

============ Fold 0 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001328 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2661
[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 92.455726
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [84]:
# 당진태양광 예측 모델 학습
capacity = 1000
LGBM_dangjin_pred = kfold_lgbm(params_dangjin, 5, energy, dangjin_fcst, dangjin_test, target='dangjin')

============ Fold 0 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002001 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4617
[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 139.910085
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

============ Fold 1 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001376 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4616
[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 140.253783
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

============ Fold 2 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001415 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4620
[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 139.207312
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [85]:
# 울산태양광 예측 모델 학습
capacity = 500
LGBM_ulsan_pred = kfold_lgbm(params_ulsan, 5, energy, ulsan_fcst, ulsan_test, target='ulsan')

============ Fold 0 ============

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000933 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1662
[LightGBM] [Info] Number of data points in the train set: 20486, number of used features: 12
[LightGBM] [Info] Start training from score 64.549253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	valid_0's l1: 13.7285	valid_0's score: 6.21853
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1666
[LightGBM] [Info] Number of data points in the train set: 20487, number of used features: 12
[LightGBM] [Info] Start training from score 63.927417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [86]:
dangjin_test = test_datast(dangjin_fcst)
ulsan_test = test_datast(ulsan_fcst)

In [133]:
LGBM_dangjin_floating_pred = LGBM_dangjin_floating_pred.reshape(672,1)
LGBM_dangjin_warehouse_pred = LGBM_dangjin_warehouse_pred.reshape(672,1)
LGBM_dangjin_pred = LGBM_dangjin_pred.reshape(672,1)
LGBM_ulsan_pred = LGBM_ulsan_pred.reshape(672,1)

In [160]:
LGBM_dangjin_floating_pred_ = LGBM_dangjin_floating_pred / c_floating
LGBM_dangjin_warehouse_pred_ = LGBM_dangjin_warehouse_pred / c_warehouse
LGBM_dangjin_pred_ = LGBM_dangjin_pred / c_dangjin
LGBM_ulsan_pred_ = LGBM_ulsan_pred / c_ulsan

## 예측결과 결합

In [161]:
new_dangjin_floating_pred = np.array([floating_pred, LGBM_dangjin_floating_pred_])
new_dangjin_warehouse_pred = np.array([warehouse_pred, LGBM_dangjin_warehouse_pred_])
new_dangjin_pred = np.array([dangjin_pred, LGBM_dangjin_pred_])
new_ulsan_pred = np.array([ulsan_pred, LGBM_ulsan_pred_])

In [163]:
new_dangjin_floating_pred[0]

array([[5.33032164e-07],
       [3.72745802e-07],
       [3.62714445e-07],
       [6.04707168e-07],
       [1.57530440e-06],
       [8.55383132e-06],
       [1.36852264e-04],
       [5.72466850e-03],
       [9.59913731e-02],
       [2.04554498e-01],
       [3.14379364e-01],
       [3.83238226e-01],
       [4.13631886e-01],
       [3.89954180e-01],
       [3.33095074e-01],
       [2.30356902e-01],
       [8.95924270e-02],
       [7.66181946e-03],
       [1.39036775e-03],
       [2.21043825e-04],
       [3.68139445e-05],
       [7.06923447e-06],
       [1.73223839e-06],
       [6.15353315e-07],
       [3.22643388e-07],
       [2.29453107e-07],
       [2.28391656e-07],
       [3.63840059e-07],
       [1.01062540e-06],
       [8.91053878e-06],
       [1.56342983e-04],
       [9.28398967e-03],
       [2.16572165e-01],
       [4.10494357e-01],
       [5.52178860e-01],
       [6.15268707e-01],
       [6.34682715e-01],
       [6.01525664e-01],
       [4.94549721e-01],
       [3.17058921e-01],


## 앙상블 모델 예측

In [164]:
# ens_model_dangjin_floating = Sequential() # 당진수상 앙상블 모델
# ens_model_dangjin_floating.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# ens_model_dangjin_floating.add(Dropout(0.3))
# ens_model_dangjin_floating.add(Dense(1, activation='sigmoid'))
# ens_model_dangjin_floating.compile(loss='mean_squared_error', optimizer='adam')
# ens_model_dangjin_floating.summary()

ens_model_dangjin_floating = Sequential() # 당진수상 앙상블 모델
ens_model_dangjin_floating.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
ens_model_dangjin_floating.add(Dropout(0.3))
ens_model_dangjin_floating.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
ens_model_dangjin_floating.add(Dense(1, activation='sigmoid'))
ens_model_dangjin_floating.compile(loss='mean_squared_error', optimizer='adam')
ens_model_dangjin_floating.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 24, 24)            3360      
_________________________________________________________________
dropout_10 (Dropout)         (None, 24, 24)            0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [165]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
ens_history_dangjin_floating = ens_model_dangjin_floating.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
524/526 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00001: val_loss improved from inf to 0.01395, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 10s 20ms/step - loss: 0.0349 - val_loss: 0.0140
Epoch 2/50
524/526 [============================>.] - ETA: 0s - loss: 0.0125- ETA: 0s - loss: 0.
Epoch 00002: val_loss improved from 0.01395 to 0.01142, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 10s 19ms/step - loss: 0.0125 - val_loss: 0.0114
Epoch 3/50
526/526 [==============================] - ETA: 0s - loss: 0.0108
Epoch 00003: val_loss improved from 0.01142 to 0.01043, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 

Epoch 32/50
522/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00032: val_loss did not improve from 0.00756
526/526 [==============================] - 7s 13ms/step - loss: 0.0081 - val_loss: 0.0078
Epoch 33/50
523/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00033: val_loss did not improve from 0.00756
526/526 [==============================] - 7s 14ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 34/50
522/526 [============================>.] - ETA: 0s - loss: 0.0083
Epoch 00034: val_loss did not improve from 0.00756
526/526 [==============================] - 7s 13ms/step - loss: 0.0083 - val_loss: 0.0086
Epoch 35/50
526/526 [==============================] - ETA: 0s - loss: 0.0081
Epoch 00035: val_loss did not improve from 0.00756
526/526 [==============================] - 7s 13ms/step - loss: 0.0081 - val_loss: 0.0091
Epoch 36/50
524/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00036: val_loss did not improve from 0.0

In [166]:
ensemble_pred_dangjin_floating = ens_model_dangjin_floating.predict(new_dangjin_floating_pred)
ensemble_pred_dangjin_floating

array([[6.44564352e-07],
       [3.77740037e-07],
       [4.52859410e-07],
       [1.25748215e-06],
       [6.15371800e-06],
       [3.85119056e-05],
       [4.18633223e-04],
       [7.42959976e-03],
       [8.30721855e-02],
       [1.87991440e-01],
       [2.94439733e-01],
       [3.98430526e-01],
       [4.47452039e-01],
       [4.51328427e-01],
       [3.30287039e-01],
       [1.43821120e-01],
       [4.65633869e-02],
       [3.41850519e-03],
       [6.05762005e-04],
       [9.74141658e-05],
       [1.50110955e-05],
       [2.61647506e-06],
       [5.45603484e-07],
       [1.57543880e-07],
       [6.55941292e-08],
       [4.50369342e-08],
       [6.11921323e-08],
       [1.24520540e-07],
       [8.19056424e-07],
       [1.05677809e-05],
       [2.41935253e-04],
       [9.19941068e-03],
       [1.86594754e-01],
       [4.20158356e-01],
       [5.37663639e-01],
       [6.06510401e-01],
       [6.36264265e-01],
       [6.27830625e-01],
       [5.26825070e-01],
       [3.32532853e-01],


In [167]:
# ens_model_dangjin_warehouse = Sequential() # 당진자재창고 앙상블 모델
# ens_model_dangjin_warehouse.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# ens_model_dangjin_warehouse.add(Dropout(0.3))
# ens_model_dangjin_warehouse.add(Dense(1, activation='sigmoid'))
# ens_model_dangjin_warehouse.compile(loss='mean_squared_error', optimizer='adam')
# ens_model_dangjin_warehouse.summary()

ens_model_dangjin_warehouse = Sequential() # 당진자재창고 앙상블 모델
ens_model_dangjin_warehouse.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
ens_model_dangjin_warehouse.add(Dropout(0.3))
ens_model_dangjin_warehouse.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
ens_model_dangjin_warehouse.add(Dense(1, activation='sigmoid'))
ens_model_dangjin_warehouse.compile(loss='mean_squared_error', optimizer='adam')
ens_model_dangjin_warehouse.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 24, 24)            3360      
_________________________________________________________________
dropout_11 (Dropout)         (None, 24, 24)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [168]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
ens_history_dangjin_warehouse = ens_model_dangjin_warehouse.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
524/526 [============================>.] - ETA: 0s - loss: 0.0333
Epoch 00001: val_loss improved from inf to 0.01470, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 12s 23ms/step - loss: 0.0333 - val_loss: 0.0147
Epoch 2/50
525/526 [============================>.] - ETA: 0s - loss: 0.0138
Epoch 00002: val_loss improved from 0.01470 to 0.01241, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 11s 22ms/step - loss: 0.0138 - val_loss: 0.0124
Epoch 3/50
526/526 [==============================] - ETA: 0s - loss: 0.0115
Epoch 00003: val_loss improved from 0.01241 to 0.01027, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - loss

526/526 [==============================] - 10s 19ms/step - loss: 0.0082 - val_loss: 0.0077
Epoch 32/50
524/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00032: val_loss did not improve from 0.00771
526/526 [==============================] - 7s 13ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 33/50
525/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00033: val_loss did not improve from 0.00771
526/526 [==============================] - 7s 13ms/step - loss: 0.0081 - val_loss: 0.0080
Epoch 34/50
525/526 [============================>.] - ETA: 0s - loss: 0.0082
Epoch 00034: val_loss did not improve from 0.00771
526/526 [==============================] - 7s 13ms/step - loss: 0.0082 - val_loss: 0.0082
Epoch 35/50
524/526 [============================>.] - ETA: 0s - loss: 0.0081
Epoch 00035: val_loss did not improve from 0.00771
526/526 [==============================] - 7s 13ms/step - loss: 0.0081 - val_loss: 0.0089
Epoch 36/50
525/526 [============

In [169]:
ensemble_pred_dangjin_warehouse = ens_model_dangjin_warehouse.predict(pred_feature)
ensemble_pred_dangjin_warehouse

array([[1.09917462e-04],
       [7.74642176e-05],
       [7.66136654e-05],
       [1.06488726e-04],
       [2.30163336e-04],
       [7.59512186e-04],
       [3.30165029e-03],
       [1.94531977e-02],
       [1.03516102e-01],
       [1.81294352e-01],
       [2.60311067e-01],
       [3.12809765e-01],
       [3.42081875e-01],
       [3.44933659e-01],
       [2.97487825e-01],
       [2.25180507e-01],
       [1.10370934e-01],
       [3.46179605e-02],
       [1.48333311e-02],
       [5.59070706e-03],
       [2.08550692e-03],
       [8.13186169e-04],
       [3.19808722e-04],
       [1.37895346e-04],
       [7.87343961e-05],
       [6.86575804e-05],
       [8.25656753e-05],
       [1.37388706e-04],
       [3.31163406e-04],
       [1.16053224e-03],
       [5.23233414e-03],
       [3.71665359e-02],
       [1.57087296e-01],
       [2.98178762e-01],
       [4.05439019e-01],
       [4.77049202e-01],
       [5.18990517e-01],
       [4.96139348e-01],
       [4.29704726e-01],
       [3.10520768e-01],


In [170]:
# ens_model_dangjin = Sequential() # 당진 앙상블 모델
# ens_model_dangjin.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# ens_model_dangjin.add(Dropout(0.3))
# ens_model_dangjin.add(Dense(1, activation='sigmoid'))
# ens_model_dangjin.compile(loss='mean_squared_error', optimizer='adam')
# ens_model_dangjin.summary()

ens_model_dangjin = Sequential() # 당진 앙상블 모델
ens_model_dangjin.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
ens_model_dangjin.add(Dropout(0.3))
ens_model_dangjin.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
ens_model_dangjin.add(Dense(1, activation='sigmoid'))
ens_model_dangjin.compile(loss='mean_squared_error', optimizer='adam')
ens_model_dangjin.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 24, 24)            3360      
_________________________________________________________________
dropout_12 (Dropout)         (None, 24, 24)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [171]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
ens_history_dangjin = ens_model_dangjin.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
523/526 [============================>.] - ETA: 0s - loss: 0.0343
Epoch 00001: val_loss improved from inf to 0.01518, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 12s 23ms/step - loss: 0.0342 - val_loss: 0.0152
Epoch 2/50
526/526 [==============================] - ETA: 0s - loss: 0.0139
Epoch 00002: val_loss improved from 0.01518 to 0.01168, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 11s 22ms/step - loss: 0.0139 - val_loss: 0.0117
Epoch 3/50
526/526 [==============================] - ETA: 0s - loss: 0.0116- ETA: 1s - loss: - ETA: 0s - 
Epoch 00003: val_loss improved from 0.01168 to 0.01038, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [========================

In [172]:
ensemble_pred_dangjin = ens_model_dangjin.predict(pred_feature)
ensemble_pred_dangjin

array([[3.90691071e-08],
       [1.83247035e-08],
       [1.96049790e-08],
       [4.90732255e-08],
       [3.62613463e-07],
       [5.97065400e-06],
       [1.80780888e-04],
       [7.64784217e-03],
       [7.79843926e-02],
       [1.93145305e-01],
       [3.10080856e-01],
       [4.00171369e-01],
       [4.43235278e-01],
       [4.22198921e-01],
       [3.25826734e-01],
       [2.09167004e-01],
       [7.38385022e-02],
       [1.30450130e-02],
       [4.03609872e-03],
       [8.47786665e-04],
       [1.12709233e-04],
       [1.13212918e-05],
       [1.87757644e-06],
       [4.09461222e-07],
       [1.43590825e-07],
       [8.44515000e-08],
       [1.18323278e-07],
       [4.66480486e-07],
       [5.68425094e-06],
       [1.38521194e-04],
       [5.68017364e-03],
       [3.99725139e-02],
       [2.31720984e-01],
       [3.91181231e-01],
       [5.00486612e-01],
       [5.55636168e-01],
       [5.71624100e-01],
       [5.43742061e-01],
       [4.61705595e-01],
       [2.82662809e-01],


In [173]:
# ens_model_ulsan = Sequential() # 울산 앙상블 모델
# ens_model_ulsan.add(LSTM(24, 
#                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
#                activation='relu', 
#                kernel_regularizer=tf.keras.regularizers.l2(0.001),
#                return_sequences=True)
#           )

# # model.add(Dense(1))
# ens_model_ulsan.add(Dropout(0.3))
# ens_model_ulsan.add(Dense(1, activation='sigmoid'))
# ens_model_ulsan.compile(loss='mean_squared_error', optimizer='adam')
# ens_model_ulsan.summary()

ens_model_ulsan = Sequential() # 울산 앙상블 모델
ens_model_ulsan.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
ens_model_ulsan.add(Dropout(0.3))
ens_model_ulsan.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
ens_model_ulsan.add(Dense(1, activation='sigmoid'))
ens_model_ulsan.compile(loss='mean_squared_error', optimizer='adam')
ens_model_ulsan.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 24, 24)            3360      
_________________________________________________________________
dropout_13 (Dropout)         (None, 24, 24)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [174]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
ens_history_ulsan = ens_model_ulsan.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
526/526 [==============================] - ETA: 0s - loss: 0.0340
Epoch 00001: val_loss improved from inf to 0.01394, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 12s 22ms/step - loss: 0.0340 - val_loss: 0.0139
Epoch 2/50
526/526 [==============================] - ETA: 0s - loss: 0.0133- ET
Epoch 00002: val_loss improved from 0.01394 to 0.01130, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 12s 22ms/step - loss: 0.0133 - val_loss: 0.0113
Epoch 3/50
524/526 [============================>.] - ETA: 0s - loss: 0.0114
Epoch 00003: val_loss improved from 0.01130 to 0.01032, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - 

In [175]:
ensemble_pred_ulsan = ens_model_ulsan.predict(pred_feature)
ensemble_pred_ulsan

array([[1.23454829e-05],
       [7.80957180e-06],
       [6.60593105e-06],
       [9.13013173e-06],
       [2.01599723e-05],
       [6.83594553e-05],
       [4.68522310e-04],
       [5.45570254e-03],
       [6.40533268e-02],
       [1.67365938e-01],
       [2.39232719e-01],
       [3.47751677e-01],
       [4.07211542e-01],
       [3.98099184e-01],
       [3.23638201e-01],
       [1.93478435e-01],
       [6.70277476e-02],
       [9.59095359e-03],
       [3.75694036e-03],
       [1.36744976e-03],
       [4.48673964e-04],
       [1.46180391e-04],
       [5.14585008e-05],
       [2.16968765e-05],
       [1.17296950e-05],
       [8.94994446e-06],
       [1.01661444e-05],
       [1.50281548e-05],
       [3.78402765e-05],
       [2.38865614e-04],
       [5.51876426e-03],
       [6.71801567e-02],
       [2.11698204e-01],
       [4.22170430e-01],
       [5.29451907e-01],
       [5.82709014e-01],
       [5.85767210e-01],
       [5.41841149e-01],
       [4.48559403e-01],
       [3.14445436e-01],


In [ ]:
day = 50
day_range = 10

plt.figure(figsize=(20, 5))
plt.title('Dangjin', fontsize=20)
plt.plot(test_label[-day*24:-(day-day_range)*24], label = 'actual')
plt.plot(pred[-day*24:-(day-day_range)*24], label = 'prediction')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(floating_pred * c_floating, label = 'dangjin_floating_pred')
plt.plot(warehouse_pred * c_warehouse, label = 'dangjin_warehouse_pred')
plt.plot(dangjin_pred * c_dangjin, label = 'dangjin_pred')
plt.plot(ulsan_pred * c_ulsan, label = 'ulsan_pred')
plt.legend()
plt.show()

In [176]:
submission = pd.read_csv('data/sample_submission.csv')

In [178]:
submission.iloc[:24*28, 1] = ensemble_pred_dangjin_floating * c_floating
submission.iloc[:24*28, 2] = ensemble_pred_dangjin_warehouse * c_warehouse
submission.iloc[:24*28, 3] = ensemble_pred_dangjin * c_dangjin
submission.iloc[:24*28, 4] = ensemble_pred_ulsan * c_ulsan

In [179]:
submission.to_csv('data/submission(21.6.25.(금) LSTM+LGBM).csv', index=False)
submission.head(48)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.000645,0.076942,0.000039,0.006173
1,2021-02-01 02:00:00,0.000378,0.054225,0.000018,0.003905
2,2021-02-01 03:00:00,0.000453,0.053630,0.000020,0.003303
3,2021-02-01 04:00:00,0.001257,0.074542,0.000049,0.004565
4,2021-02-01 05:00:00,0.006154,0.161114,0.000363,0.010080
5,2021-02-01 06:00:00,0.038512,0.531659,0.005971,0.034180
6,2021-02-01 07:00:00,0.418633,2.311155,0.180781,0.234261
7,2021-02-01 08:00:00,7.429600,13.617238,7.647842,2.727851
8,2021-02-01 09:00:00,83.072189,72.461273,77.984390,32.026665
9,2021-02-01 10:00:00,187.991440,126.906044,193.145309,83.682968


In [ ]:
import pandas as pd
import numpy as np

def sola_nmae(answer_df, submission_df):
    submission = submission_df[submission_df['time'].isin(answer_df['time'])]
    submission.index = range(submission.shape[0])
    
    # 시간대별 총 발전량
    sum_submission = submission.iloc[:,1:].sum(axis=1)
    sum_answer = answer_df.iloc[:,1:].sum(axis=1)
    
    # 발전소 발전용량
    capacity = {
        'dangjin_floating':1000, # 당진수상태양광 발전용량
        'dangjin_warehouse':700, # 당진자재창고태양광 발전용량
        'dangjin':1000, # 당진태양광 발전용량
        'ulsan':500 # 울산태양광 발전용량
    }
    
    # 총 발전용량
    total_capacity = np.sum(list(capacity.values()))
    
    # 총 발전용량 절대오차
    absolute_error = (sum_answer - sum_submission).abs()
    
    # 발전용량으로 정규화
    absolute_error /= total_capacity
    
    # 총 발전용량의 10% 이상 발전한 데이터 인덱스 추출
    target_idx = sum_answer[sum_answer>=total_capacity*0.1].index
    
    # NMAE(%)
    nmae = 100 * absolute_error[target_idx].mean()
    
    return nmae